In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import re
import warnings
from sklearn.base import BaseEstimator, TransformerMixin
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import make_pipeline, Pipeline
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import joblib
import logging

In [ ]:
train = pd.read_csv('/content/data/train.csv')
test = pd.read_csv('/content/data/test_features.csv')

In [ ]:
train


,ID,Text,Category
0,969,@JuliaBradbury @SimonCalder @walsop @HodderPRI...,0
1,241,or here https://t.co/R2tO79Easn … .An in house...,1
2,820,@britshmuseum @thehistoryguy Gosh periscope is...,2
3,693,@Ophiolatrist britishmuseum The stupid #French...,1
4,421,@SassyClde We won't stop til @britishmuseum du...,1
...,...,...,...
1595,1131,@tateliverpool Great stuff! I'll be there ASAP...,0
1596,1295,View of L’Estaque by Cézanne atrisk of being e...,3
1597,861,Agreed @rhiannonakelly. Eat my sandwiches @bri...,1
1598,1460,"@BAGcurators @NationalGallery Yes, it is. It'...",2


In [ ]:
test

,ID,Text
0,1861,Goodbye @kettlesyard see you in .25 years! htt...
1,354,"@BBC_Culture @PlymouthMuseum Oh dear, why not ..."
2,1334,Fantastic @johnmcdonnellMP standing up for wor...
3,906,"@BBC_Culture @PlymouthMuseum Oh dear, why not ..."
4,1290,@britishmuseum @TripAdvisor it is !
...,...,...
395,966,Unfortunately the transition narrative parts o...
396,1285,#AskTheGallery Love @NationalGallery. Hate Pri...
397,1740,"@CelesteRegal Well, I hope so - but I don't su..."
398,262,@tateliverpool I'm trying to book the members ...


In [ ]:
class TextProcessor(BaseEstimator, TransformerMixin):
    def __init__(self, lower=False, stem=False):
        self.lower = lower
        self.stem = stem

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        def text_processing(text):
            processed_text = re.sub('[^a-zA-Z]', ' ', text) # this code removes any non-alphabet characters
            if self.lower:
                processed_text = processed_text.lower()
            processed_text = processed_text.split()
            if self.stem:
                ps = PorterStemmer()
                processed_text = [ps.stem(word) for word in processed_text if word not in set(stopwords.words('english'))]
            processed_text = ' '.join(processed_text)
            return processed_text

        return [text_processing(text) for text in X]

In [ ]:
from sklearn.linear_model import LogisticRegression
vectorizer = CountVectorizer(max_features=3000)
text_processor = TextProcessor(lower=True, stem=False)
logistics = LogisticRegression(random_state=42, max_iter=605, C=0.7966976438448329)

logistics_pipeline = Pipeline(
    [
        ("text_processing", text_processor),
        ("vectorizer", vectorizer),
        ("logistics", logistics),
    ]
)

logistics_pipeline.fit(train['Text'], train['Category'])

Pipeline(steps=[('text_processing', TextProcessor(lower=True)),
                ('vectorizer', CountVectorizer(max_features=3000)),
                ('logistics',
                 LogisticRegression(C=0.7966976438448329, max_iter=605,
                                    random_state=42))])

In [ ]:
train_pred = logistics_pipeline.predict(train['Text'])
train_accuracy = accuracy_score(train['Category'], train_pred)
print("Train set accuracy:", train_accuracy)

Train set accuracy: 1.0


In [ ]:
test_pred = logistics_pipeline.predict(test['Text'])


In [ ]:
test['Category'] = test_pred


In [ ]:
test[['ID', 'Category']].to_csv('submission.csv', index=False)
